In [1]:
import os
import json
import pandas as pd
import numpy as np
import tqdm
import scipy.sparse as sp

import implicit
import lightfm
import warnings
warnings.filterwarnings('ignore')

In [2]:
pd.set_option('display.max_columns',100)

DATA_PATH = '../okko/orig_data'
PREPARED_PATH = './prepared_data/'

In [3]:
actions = pd.read_pickle(PREPARED_PATH+'actions_one_table.pkl')

In [4]:
actions.sort_index(inplace = True) # На всякий случай, иначе деление не будет работать

In [5]:
actions.head()

action consumption_mode  \
user_uid element_uid ts                                     
0        51          4.416546e+07  watch                S   
         72          4.375829e+07  watch                S   
         207         4.371904e+07  watch                S   
         209         4.377814e+07  watch                S   
         434         4.338109e+07  watch                S   

                                   device_manufacturer  device_type  rating  \
user_uid element_uid ts                                                       
0        51          4.416546e+07                 99.0          0.0     NaN   
         72          4.375829e+07                 99.0          0.0     NaN   
         207         4.371904e+07                 99.0          0.0     NaN   
         209         4.377814e+07                 99.0          0.0     NaN   
         434         4.338109e+07                 99.0          0.0     NaN   

                                   watched_time  duration  type  
user_uid element_uid ts                                          
0        51          4.416546e+07       12382.0      3600     1  
         72          4.375829e+07        5653.0      6000     1  
         207         4.371904e+07        2646.0      5400     1  
         209         4.377814e+07        6971.0      7200     1  
         434         4.338109e+07        5894.0      6600     1

In [6]:
def get_train_test(actions,mode = 'by_time',perc = (0.6,0.2,0.2)):
    '''
    здесь не очень аккуратно обращаемся с временем просмотра, потому что фильмы на границе должны быть 
    с обрезанной длительностью - но насрать
    '''
    X = actions.copy()
    if mode == 'by_time':
        X['ones'] = 1
        X['increment'] = np.arange(len(X))
        by_time = X.groupby(level = 2)['ones'].sum()
        by_time.sort_index(inplace = True)
        #проверили, что вроде как все ок и равномерно во времени
        cur = 0
        idx = []
        for i in range(len(perc)):
#             print(np.round((cur)*len(by_time)),np.round((cur+perc[i])*len(by_time)))
            by_time_temp = by_time.iloc[int(np.round((cur)*len(by_time))):int(np.round((cur+perc[i])*len(by_time)))].index.values
            print(len(by_time_temp))
            mn = by_time_temp.min()
            mx = by_time_temp.max()
            cur+=perc[i]
            idx.append(X.loc[(slice(None),slice(None),slice(mn,mx)),'increment'].values)
            
        return idx

In [7]:
idx = get_train_test(actions)

6558458
2186152
2186153


In [8]:
actions.iloc[idx[0]].index.get_level_values(2).max()

43362401.96226887

In [9]:
actions.iloc[idx[1]].index.get_level_values(2).min()

43362401.97085199

In [10]:
actions.iloc[idx[1]].index.get_level_values(2).max()

43828341.47903843

In [11]:
actions.iloc[idx[2]].index.get_level_values(2).min()

43828341.48519237

In [12]:
actions.consumption_mode.value_counts()

S    8296227
P     873834
R     472951
Name: consumption_mode, dtype: int64

In [13]:
actions.head()

action consumption_mode  \
user_uid element_uid ts                                     
0        51          4.416546e+07  watch                S   
         72          4.375829e+07  watch                S   
         207         4.371904e+07  watch                S   
         209         4.377814e+07  watch                S   
         434         4.338109e+07  watch                S   

                                   device_manufacturer  device_type  rating  \
user_uid element_uid ts                                                       
0        51          4.416546e+07                 99.0          0.0     NaN   
         72          4.375829e+07                 99.0          0.0     NaN   
         207         4.371904e+07                 99.0          0.0     NaN   
         209         4.377814e+07                 99.0          0.0     NaN   
         434         4.338109e+07                 99.0          0.0     NaN   

                                   watched_time  duration  type  
user_uid element_uid ts                                          
0        51          4.416546e+07       12382.0      3600     1  
         72          4.375829e+07        5653.0      6000     1  
         207         4.371904e+07        2646.0      5400     1  
         209         4.377814e+07        6971.0      7200     1  
         434         4.338109e+07        5894.0      6600     1

In [14]:
# Вроде не пересекается.
train,test,valid = actions.iloc[idx[0]],actions.iloc[idx[1]],actions.iloc[idx[2]]

In [15]:
def get_target(actions):
    '''
    Функция, которая вернет число просмотреннх серий каждым пользователем каждого сериала, потом вернет то,что недопотребил
    А потом то, что точно потребил согласно правилам соревнования - например, так можно вычислить примерную длительность сериала 
    и его же рекомендовать в потребленные после.
    '''
    watch_actions = actions[actions.action == 'watch']
    # Блок нахождения всяких статистик по сериалам
    serials = watch_actions[watch_actions['type'] != 1]
    # Заменим длиетльность на 0, там где длительности нет.. или это очень короткие, надо подумоть.
    serials['num_of_series'] = (serials['watched_time']/serials['duration']).fillna(0).replace(np.inf,0).astype(int)
    serials['time_being'] = serials.index.get_level_values(2)
    dur_being = serials.groupby(level = 1).agg({'time_being':[min,len],'num_of_series':[lambda x:x.mode()[0],max]})
    dur_being.columns = ['time_being','count_of_watch','num_of_series_mode','num_of_series_max']
    # Модифицируем длитеьность сериала - как произвелдение числа серий на продолжиттельность одной
    dur = watch_actions.join(dur_being['num_of_series_max'])['num_of_series_max']*watch_actions['duration']
    watch_actions.loc[~dur.isnull(),'duration'] = dur[~dur.isnull()]
    
    
    # Блок нахождения статистик по фильмам для пользователя
    films = watch_actions[watch_actions['type'] == 1]
    # Здесь важно видимо, как долго смотрел
    films['time_being'] = films.index.get_level_values(2)
    dur_films = films.groupby(level = 1).agg({'time_being':[min,len]})
    dur_films.columns = ['time_being','count_of_watch']
    
    # Блок нахождения статистик по фильмам и пользователям
    watch_actions['rel_dur'] = (watch_actions['watched_time']/watch_actions['duration'])
    target = 1*(watch_actions['rel_dur'] >= 1/3) | watch_actions['consumption_mode'].isin(['R','P']) 
    target = target.groupby(level = [0,1]).mean()
    watch_actions = watch_actions.groupby(level = [0,1]).mean()
    watch_actions['rel_dur'] = watch_actions['rel_dur'].replace(np.inf,1)# Заглушка для фильмов с 0 длительностью
    
    
    
    return dur_being,dur_films,watch_actions,target

In [16]:
%time 
dur_being_train,dur_films_train,watch_actions_train,target_train = get_target(train)
dur_being_test,dur_films_test,watch_actions_test,target_test = get_target(test)
dur_being_valid,dur_films_valid,watch_actions_valid,target_valid = get_target(valid)

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 6.44 µs


In [20]:
#watch_actions_train.loc[watch_actions_train['type']!=1].head()

In [21]:
# Получили фичи для фильмов
import pickle
with open(PREPARED_PATH+'catalogue_features.pkl','rb') as f:
    match_element_row,match_row_element,match_columns,element_matrix = pickle.load(f)

In [22]:
from sklearn.base import TransformerMixin

In [23]:
import pickle
from scipy.sparse import coo_matrix,vstack,hstack
from sklearn.feature_extraction.text import CountVectorizer

with open(PREPARED_PATH+'bag_of_attr_movie.pkl','rb') as f:
    bag_of_attr = pickle.load(f)


class FeatureExtractor(TransformerMixin):
    def __init__(self,all_about_movie,bag_of_attr,is_censor = True,delimiter = 4, mode = 'raiting',target_col_name = 'rating'):
        self.all_about_movie = all_about_movie
        self.movie_attr_matrix = all_about_movie['movie_attr_matrix']
        self.movie_match_columns_attr = all_about_movie['movie_match_columns']
        self.movie_match_attr_columns = all_about_movie['movie_match_columns']
        self.movie_match_row_movie = all_about_movie['movie_match_row_movie']
        self.movie_match_movie_row = all_about_movie['movie_match_movie_row']

        self.is_censor = is_censor
        self.delimiter = delimiter
        self.mode = mode
    def fit(self,X):
        watch_actions_train = X[X['action'] == 'watch']
        # Сначала нам нужна матрица из всех просмотренных фильмов в трейне и меппинги оттуда 
        # - это исчерпывающая информация известная на конец трейна
        res = get_users_features(watch_actions_train,bag_of_attr)
        
        self.match_user_row = res[0] 
        self.match_row_user = res[1]
        self.match_feature_columns = res[2]
        self.match_columns_feature = res[3]
        self.train_user = res[4]
        # Вообще фильмов здесь намного больше дб, наверное стоит как-то смеппить признак 1
        # но для обычных рекомендаций это не так уж и важно.
        
        
        
        
        return self
    def transform(self,X,y = None,):
        if mode == 'rating':
            part_of_train = X.loc[X.action =='rate',target_col_name].groupby(level = [0,1]).mean().to_frame() 
        elif mode == 'duration':
            part_of_train = X.loc[X.action =='watch',target_col_name].groupby(level = [0,1]).mean().to_frame()
        res = df_to_matrix(part_of_train,self.match_user_row,self.match_element_row,self.delimiter)
        return res
class ColdFeatureExtractor(TransformerMixin):
    def __init__(self,fitted_FE):
        self.fitted_FE = fitted_FE
        self.im_columns = ['is_purchase',
             'is_rent',
             'is_subscription',
             'duration',
             'feature_1',
             'feature_2',
             'feature_3',
             'feature_4',
             'feature_5',
             'type_movie',
             'type_serial',]
        
    def fit(self,X):
        # Задача вычленить фильмы из трейна из большой матрицы фильмов и перенумеровать id
        # Здесь же когда-нибудь появтся новинки
        self.movie_train = np.unique(X[X['action'] == 'watch'].index.get_level_values(1))
        
        # Теперь нужна матрица атрибутов фильмов для юзера
        self.attr_train_map = list(self.fitted_FE.match_feature_columns.keys())
        
        self.train_movie_rows = [self.fitted_FE.movie_match_movie_row[i] for i in self.movie_train]
        self.train_movie_cols = [self.fitted_FE.movie_match_attr_columns[i] for i in  self.attr_train_map]
        
        
        
        return self
    def transform(self,X,y = None,):
        # Сначала надо получить список не новинок, доступных на конец трейна
        res = get_cold_start_matrix(actions,match_user_row,match_feature_columns,match_movie_columns):
        
        # Теперь набор атрибутов и 
        return res

In [ ]:
# Получили фичи для юзеров (пока какие-то)
# import pickle
# with open(PREPARED_PATH+'catalogue_users.pkl','rb') as f:
#     match_element_row_user,match_row_element_user,match_columns_user,element_matrix_user = pickle.load(f)
# Кажется, что их правильно пересчитывать по тем, кто есть во времени сейчас. 

# Итак, нам надо вопроизвести максимально похоже условия использования системы. т.е. на момент времени t_train_end
# мы имеем только фильмы из трейна. и атрибуты от фильмов из трейна.
# теперь в момент t_test_end  мы будем иметь N  новых фильмов и M  новых пользователей - это задачи холодного старта.
# Разобьем нашу задачу на 4 и правильно сформируем тест.
# 1- старые пользователи - старые фильмы
# 2 - новые пользователи - старые фильмы
# 3 - старые пользователи - новые фильмы
# 4 - новые пользователи - новые фильмы


def get_users_features(actions,bag_of_attr):
    '''
    Получаем трейн
    bag_of_attr - словарь, где просто каждому id  фильма сопоставлена строка атрибутов через запятую.
    строго  говоря в просмотренных фильмах атрибутов может оказаться меньше, чем во всем пуле фильмов, но я 
    пока не знаю проблема ли это ToDo
    Если history_movie определен из теста, например, то мы должны убирать новинки из формирования матрицы для простого обучения.
    Без холодного старта.
    '''
    # Приделаем каждому чуваку атрибуты просмотренных фильмов. ну или вообще по всем действиям - они все позитивные
    ind_user = []
    buf = []
    for i in tqdm.tqdm(np.unique(actions.index.get_level_values(0))):
        
        temp = np.unique(actions.loc[i].index.get_level_values(0))
        ind_user.append(i)

        s = ''
        for ii in temp:
#             if (history_movie is None) or (ii in list(history_movie.keys())):
                s+=bag_of_attr[ii]

                s+=','
        #assert X.shape[1] == len(a)
        buf.append(s)

    cv1 = CountVectorizer(token_pattern='\d+',)
    X_user = cv1.fit_transform(buf)
    
    match_user_row = {i:ii for ii,i in enumerate(ind_user)}
    match_row_user = {ii:i for ii,i in enumerate(ind_user)}
    match_feature_columns = {i:ii for ii,i in enumerate(list(cv1.get_feature_names()))}
    match_columns_feature = {ii:i for ii,i in enumerate(list(cv1.get_feature_names()))}
    print(X_user.shape,len(match_user_row),len(match_feature_columns))
    return match_user_row,match_row_user,match_feature_columns,match_columns_feature,X_user
def shape_corrector(X,num_col,num_row):
    if X.shape[0]<num_row:
        X = vstack((X,coo_matrix((int(num_row - X.shape[0]),X.shape[1]))))
    if X.shape[1]<num_col:
        
        X = hstack((X,coo_matrix((X.shape[0],int(num_col - X.shape[1])))))
    return X
def get_cold_start_matrix(actions,match_user_row,match_feature_columns,match_movie_columns):
    '''
    Нужно переписать через coo_matrix, чтоб все атрибуты совпадали
    '''
    # Наполнение по тесту для старых пользователей и старых фильмов
    row_ = []
    col_ = []
    ones = []
    # Наполнение матрицы по старым атрибутам для новых пользователей
    row_user = []
    col_user = []
    ones_user = []
    # Здесь id  фильмов, которые не смотрели в трейне
    new_movie_buf = []
    
    buf = []
    
    ind_user = []
    for i in tqdm.tqdm(np.unique(actions.index.get_level_values(0))):
        if i in match_user_row:
            temp = np.unique(actions.loc[i].index.get_level_values(0))


            s = ''
            for ii in temp:
                for k in bag_of_attr[ii].split(','):
                    if k in match_feature_columns:
                        row_.append(match_user_row[i])
                        col_.append(match_feature_columns[k])
                        ones.append(1)
                if ii not in match_movie_columns:
                    # Фильма нет в трейне
                    # Значит нужно просто сохранить его id и забрать из большой таблицы с фичами и атрибутами
                    new_movie_buf.append(ii)
                    
        else:
            # Пользователя не было в трейне
            # По сути надо создать еще несколько массивов и мапов
            temp = np.unique(actions.loc[i].index.get_level_values(0))
            ind_user.append(i)

            for ii in temp:
                for k in bag_of_attr[ii].split(','):
                    if k in match_feature_columns:
                        row_user.append(len(ind_user)-1)
                        col_user.append(match_feature_columns[k])
                        ones_user.append(1)
                if ii not in match_movie_columns:
                    # Фильма нет в трейне и еще нет пользователя
                    pass
                    # ХЗ че с этим делать
            

    # По построению test matrix должна иметь те же размеры, что и трейн матрикс, но тут надо быть аккуратнее
    # Вроде как если не попадется максимальный номер строки или столбца, то он его не нарастит - надо проверку бы
    test_matrix = coo_matrix((ones,(row_,col_)))# Старые юзеры, старые фильмы, но новое распределение атрибутов
    test_matrix = shape_corrector(test_matrix,max(match_feature_columns.values())+1,max(match_user_row.values())+1)
    # ToDo - мб нужно будет как-то сложить матрицу атрибутов, но вроде не надо
    
    
    new_user_matrix = coo_matrix((ones_user,(row_user,col_user)))
    new_match_user_row = {i:ii for ii,i in enumerate(ind_user)}
    new_match_row_user = {ii:i for ii,i in enumerate(ind_user)}
    new_user_matrix = shape_corrector(new_user_matrix,max(match_feature_columns.values())+1,max(new_match_user_row.values())+1)
    
    
    new_match_row_movie = {ii:i for ii,i in enumerate(new_movie_buf)}
    new_match_movie_row = {i:ii for ii,i in enumerate(new_movie_buf)}
    
    
    return test_matrix,new_match_user_row,new_match_row_user,new_user_matrix,new_match_row_movie,new_match_movie_row

In [ ]:

def df_to_matrix(X,match_user_row,match_element_row, is_censor = True, delimiter = 4):
    '''
    На вход подается датафрейм с мультииндексом <user_id, element_id> и некоторой оценкой пары, затем он переупорядочивается и дополняется 
    по шаблонам из строк всяких спарс матричек для фильмов и юзеров
    match_user_row - отображение из айди в номер строки в матрице, match_element_row - аналогично
    '''
    Y = X.copy()
    if is_censor:
        Y[(Y<delimiter)] = -1
        Y[(Y>=delimiter)] = 1
    Y['users'] = Y.index.get_level_values(0).map(match_user_row)
    Y['items'] = Y.index.get_level_values(1).map(match_element_row)
    Y.dropna(subset = ['users','items'],inplace = True)
    Y['users'] = Y['users'].astype(int)
    Y['items'] = Y['items'].astype(int)
    Z = coo_matrix((Y[X.columns].values.squeeze(),(Y['users'].values,Y['items'].values)))
    print(max(match_element_row.values())+1,max(match_user_row.values())+1)
    Z = shape_corrector(Z,max(match_element_row.values())+1,max(match_user_row.values()) +1)
    print(X.shape,Y.shape)
    print(Z.shape)
    return Z

In [ ]:
# Получим что-то сначала для трейна, причем для рейтингового
# Вообще парллелится, но пока непонятно зачем кроме тренировки
# Через рейтинги 
# match_user_row,match_row_user,match_feature_columns,match_columns_feature,train_user = get_users_features(train[train.action =='rate'],bag_of_attr)

# X = train.loc[train.action =='rate','rating'].groupby(level = [0,1]).mean().to_frame()
# Через длительность просмотра
match_user_row,match_row_user,match_feature_columns,match_columns_feature,train_user = get_users_features(watch_actions_train,bag_of_attr)

X = watch_actions_train['rel_dur'].groupby(level = [0,1]).mean().to_frame().replace(np.inf,1).fillna(0)
# X.value_counts()


In [ ]:
train_matrix = df_to_matrix(X,match_user_row,match_element_row,delimiter=1/3)
# test_matrix = df_to_matrix(test.loc[test.action =='rate','rating'].groupby(level = [0,1]).mean().to_frame(),match_user_row,match_element_row)
XX = watch_actions_test['rel_dur'].groupby(level = [0,1]).mean().to_frame().replace(np.inf,1).fillna(0)
test_matrix = df_to_matrix(XX,match_user_row,match_element_row,delimiter=1/3)

In [ ]:
test_user_matrix,new_match_user_row,new_match_row_user,new_user_matrix,new_match_row_movie,new_match_movie_row = get_cold_start_matrix(watch_actions_test,match_user_row,match_feature_columns,match_element_row)

In [ ]:
print(train_user.shape)

In [ ]:
print(test_matrix.shape,len(new_match_user_row),len(new_match_row_user),new_user_matrix.shape)

In [ ]:
# Теперь большая простынь с переименованием данных
#  Под каждую из 3 задач
# 1 стврые - старые
old_old_tr_te_1 = {'train_interactions':train_matrix,
                 'test_interactions':test_matrix,
                'user_features_train':None,
                'item_features_train':None,
                'user_features_test':None,
                'item_features_test':None,}
# 2
new_old_tr_te_2 = {'train_interactions':train_matrix,
                 'test_interactions':test_matrix,
                'user_features_train':train_user,
                'item_features_train':None,
                'user_features_test':new_user_matrix,
                'item_features_test':None,}
# 3

# Сначала надо обрезать фильмы по атрибутам, которые известны только по трейну.
need_columns = ['is_purchase',
 'is_rent',
 'is_subscription',
 'duration',
 'feature_1',
 'feature_2',
 'feature_3',
 'feature_4',
 'feature_5',
 'type_movie',
 'type_serial',]
need_columns.extend(list(match_feature_columns.keys()))
need_columns =[match_columns.index(i) for i in need_columns]

# Теперь нужны новинки из теста, и их надо достать из большой матрицы с филмами
# ToDo - там еще по идее должны переделываться match rows для всех не новинок, но я пока забью.



# element_matrix
# old_new_tr_te_3 = {'train_interactions':train_matrix,
#                  'test_interactions':test_matrix,
#                 'user_features_train':None,
#                 'item_features_train':None,
#                 'user_features_test':None,
#                 'item_features_test':None,}


In [ ]:
print(len(match_user_row),len(match_element_row),train_matrix.shape)

In [ ]:
# def fit_lightfm(train,item_features=None,seed = 0)
seed = 0
epochs = 30
num_threads=4

model = lightfm.LightFM(loss = 'warp',random_state=seed)


# model.fit(train_matrix,user_features = train_user,item_features = element_matrix,epochs = epochs,num_threads = num_threads,
#          verbose = True)


In [ ]:
tr = old_old_tr_te_1['train_interactions']
te = old_old_tr_te_1['test_interactions']
# tr[(tr < 3) &  (tr>0)] = -1
# tr[(tr > 3)] = 1
# te[(tr < 3) &  (te>0)] = -1
# te[(tr > 3)] = 1

model.fit(train_matrix,epochs = epochs,num_threads = num_threads,
         verbose = True)

In [ ]:
te.shape

In [ ]:
from lightfm.evaluation import auc_score,precision_at_k

# Compute and print the AUC score
train_auc = precision_at_k(model, te ,tr, k = 20,num_threads=4).mean()
print('Collaborative filtering train AUC: %s' % train_auc)

In [ ]:
new_old_tr_te_2.keys()

In [ ]:
tr = new_old_tr_te_2['train_interactions']
te = new_old_tr_te_2['test_interactions']
uf = new_old_tr_te_2['user_features_train']
tf = new_old_tr_te_2['user_features_test']
print(tr.shape,te.shape,uf.shape,tf.shape)
# tr[(tr < 3) &  (tr>0)] = -1
# tr[(tr > 3)] = 1
# te[(tr < 3) &  (te>0)] = -1
# te[(tr > 3)] = 1

model.fit(train_matrix,user_features=uf,epochs = epochs,num_threads =num_threads,
         verbose = True)

In [ ]:
from scipy.sparse import csr_matrix
import matplotlib.pyplot as plt
tf = csr_matrix(tf)

In [ ]:
p = model.predict(0,np.arange(tr.shape[1]),user_features=tf[0,:],num_threads=num_threads)

In [ ]:
plt.hist(p)

In [ ]:
test_user[0,:]

In [ ]:
a = model.predict(0,np.arange(train_matrix.shape[1]),user_features=test_user[0,:],num_threads=num_threads)
len(a)

In [ ]:
from lightfm.evaluation import auc_score,precision_at_k

# Compute and print the AUC score
train_auc = precision_at_k(model, train_matrix,user_features = train_user,item_features = element_matrix ,num_threads=num_threads).mean()
print('Collaborative filtering train AUC: %s' % train_auc)

In [ ]:
model.user_feature_map

In [ ]:
def get_answer(test_users_dict,test):
    pass